In [23]:
import pandas as pd
import duckdb
import requests
import tempfile
import random

import sys
sys.path.append("./src")
import env_options
import lmsys_dataset_handler as lmsys

dotenv_path = "../../../../../apis/.env"
hf_token, hf_token_write = env_options.check_env(colab=False, use_dotenv=True, dotenv_path=dotenv_path)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


### Checking Parquet file URLs

In [2]:
headers = {"Authorization": f"Bearer {hf_token}"}
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/main/data/train-00000-of-00001.parquet'
head_response = requests.head(url, allow_redirects=True, headers=headers)
file_size = int(head_response.headers['Content-Length'])
print(f"File size: {file_size} bytes")

File size: 7717425 bytes


In [3]:
for item in head_response.headers:
    print(f"{item}: {head_response.headers[item]}")

Content-Type: binary/octet-stream
Content-Length: 7717425
Connection: keep-alive
Date: Wed, 05 Mar 2025 12:15:11 GMT
Last-Modified: Sun, 23 Feb 2025 22:08:58 GMT
ETag: "fac5e3bd16eee07439827a964ce75da4"
x-amz-storage-class: INTELLIGENT_TIERING
x-amz-server-side-encryption: AES256
x-amz-version-id: 4fMBdCs62qiK1.bPMU2uFCaLbr0nSLn4
Content-Disposition: inline; filename*=UTF-8''train-00000-of-00001.parquet; filename="train-00000-of-00001.parquet";
Accept-Ranges: bytes
Server: AmazonS3
X-Cache: Hit from cloudfront
Via: 1.1 cf66796b663642e35e0aca515f0d4a6e.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: MAD56-P2
X-Amz-Cf-Id: eExMXCono3C-NZi78_un-t9nmtSBbxUcxIP3AjMfxYa5BbSEHYcXWQ==
Age: 120812
Content-Security-Policy: default-src 'none'; sandbox
Vary: Origin


### Querying Parquet files

In [4]:
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/main/data/train-00000-of-00001.parquet'
query = """
        SELECT * FROM read_parquet('{url}') LIMIT 2
        """
query = query.format(url=url)
query_result = duckdb.query(query).df()
display(query_result)


,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
0,99acfac8-2c78-4ab6-964d-5d63449fd539,1,Help me rephrase this tagline by giving me a f...,"Sure, I can help you with that. Here are some ...",Culture and Entertainment,en,request,0.976421,learn,0.956283,openai-gpt,,2023-06-01,12,bing-skype,,,[],
1,99acfac8-2c78-4ab6-964d-5d63449fd539,2,Give me a creative sentence for a closed ended...,Here are some creative sentences for a closed ...,Language and writing,en,request,0.985784,learn,0.953707,openai-gpt,,2023-06-01,12,bing-skype,,,[],


### Querying LMSYS files

In [5]:
data_files = [
    "train-00000-of-00006-4feeb3f83346a0e9.parquet",
    "train-00001-of-00006-4030672591c2f478.parquet",
    "train-00002-of-00006-1779b7cec9462180.parquet",
    "train-00003-of-00006-2fa862bfed56af1f.parquet",
    "train-00004-of-00006-18f4bdd50c103e71.parquet",
    "train-00005-of-00006-fe1acc5d10a9f0e2.parquet"
]


for file in data_files:
    url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{file}'
    head_response = requests.head(url, allow_redirects=True, headers=headers)
    file_size = int(head_response.headers['Content-Length'])
    print(f"{file}: {file_size} bytes")

train-00000-of-00006-4feeb3f83346a0e9.parquet: 249303811 bytes
train-00001-of-00006-4030672591c2f478.parquet: 247222671 bytes
train-00002-of-00006-1779b7cec9462180.parquet: 249923890 bytes
train-00003-of-00006-2fa862bfed56af1f.parquet: 247173225 bytes
train-00004-of-00006-18f4bdd50c103e71.parquet: 246443273 bytes
train-00005-of-00006-fe1acc5d10a9f0e2.parquet: 248783380 bytes


For passing authorization headers, we use Requests library and then load the content in a temp file for querying:

In [ ]:
url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{data_files[1]}'
print(url)
# Download file with auth headers using requests
r = requests.get(url, headers=headers)
# Write the downloaded content into a temporary file
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name
# Query using DuckDB from the temporary file
query_result = duckdb.query(f"SELECT * FROM read_parquet('{tmp_path}') LIMIT 2").df()
display(query_result)

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/train-00001-of-00006-4030672591c2f478.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,23828586342d4baca3cfd731650b78bc,vicuna-13b,"[{'content': 'You can say nsfw? ', 'role': 'us...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,25cfe27cf42a487aa0c2b574ffbea283,vicuna-13b,"[{'content': 'Hello, we met with my girlfriend...",3,English,"[{'categories': {'harassment': False, 'harassm...",True


In [ ]:
random.shuffle(data_files)
url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{data_files[0]}'
print(url)
r = requests.get(url, headers=headers)
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name

query = """
    SELECT * FROM read_parquet('{tmp_path}') 
    WHERE contains(lower(cast(conversation as VARCHAR)), 'mounting an');
    """
query = query.format(tmp_path=tmp_path)
query_result = duckdb.query(query).df()
display(query_result)

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/train-00001-of-00006-4030672591c2f478.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,a16d07894c234b7ab1b9e295516f09e7,guanaco-33b,[{'content': 'Write a long erotic story about ...,1,English,"[{'categories': {'harassment': False, 'harassm...",True
1,084ec5cb1f3541de943610e41a118a63,alpaca-13b,[{'content': 'But luks is supported by most di...,5,English,"[{'categories': {'harassment': False, 'harassm...",False
2,6166f58de6f24f178f0cc41a65df0081,wizardlm-13b,[{'content': 'ten steps (bulleted format) desc...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,449eff6b91fb41f0a6b786e82de8bdde,vicuna-33b,[{'content': 'Write a long descriptive explici...,5,English,"[{'categories': {'harassment': False, 'harassm...",True
4,e40bacd289d2465b971a6f6c22426a70,wizardlm-13b,[{'content': 'ten steps (bulleted format) desc...,1,English,"[{'categories': {'harassment': False, 'harassm...",True
5,0704c9998d864c36be548c43cc3690fc,vicuna-13b,[{'content': 'How do you mount and secure a ma...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
6,de4d08e142dc471994e7166cedea9eae,wizardlm-13b,[{'content': 'write a detailed short story of ...,3,English,"[{'categories': {'harassment': False, 'harassm...",True
7,7e538364543c40c687ed7d774d71c3e0,wizardlm-13b,[{'content': 'ten steps (bulleted format) desc...,2,English,"[{'categories': {'harassment': False, 'harassm...",True
8,33a63c8a262f4b60a2ac8ea47dfc61c0,vicuna-13b,[{'content': 'Help me choose one Your company...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
9,3c0ce53a59a34ec6a41bc3603b7a7da2,vicuna-13b,[{'content': 'Write a very accurate response s...,2,English,"[{'categories': {'harassment': False, 'harassm...",True


### Querying function

In [30]:
def search_in_files(filter_condition, files_list, min_results=1):
    """
    Searches through the given list of parquet files until at least min_results are found.
    
    Parameters:
    - filter_condition: SQL condition (without the WHERE keyword) to filter rows.
                        For example: "contains(lower(cast(conversation as VARCHAR)), 'mounting an')"
    - min_results: Minimum number of results to be satisfied before stopping the search.
    - files_list: List of parquet file names to search into.
    
    Returns:
    - A pandas DataFrame with the query results (possibly empty if no file meets the criteria).
    """
    files = files_list.copy()
    random.shuffle(files)
    
    for file_name in files:
        url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{file_name}'
        print(f"Querying file: {file_name}")
        r = requests.get(url, headers=headers)
        with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
            tmp.write(r.content)
            tmp_path = tmp.name
        
        query_str = f"""
            SELECT * FROM read_parquet('{tmp_path}') 
            WHERE {filter_condition}
            """
        df = duckdb.query(query_str).df()
        print(f"Found {len(df)} result(s) in {file_name}")
        
        if len(df) >= min_results:
            return df
    
    return df  # return the last result even if it doesn't meet min_results

random.shuffle(data_files)
filter_condition = "contains(lower(cast(conversation as VARCHAR)), 'b00bz')"
df = search_in_files(filter_condition=filter_condition, files_list=data_files, min_results=1)
display(df)

Querying file: train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Found 0 result(s) in train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Querying file: train-00001-of-00006-4030672591c2f478.parquet
Found 0 result(s) in train-00001-of-00006-4030672591c2f478.parquet
Querying file: train-00003-of-00006-2fa862bfed56af1f.parquet
Found 5 result(s) in train-00003-of-00006-2fa862bfed56af1f.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,c71d21e138a549e3bc510dd9ce28abd3,koala-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
2,c444480bb73d47f58e7026b7f8c95028,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
3,5f2bd20b2cde438b8d3b32e4283928ae,chatglm-6b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,70069683196b47ba9cdaac0af63be2b7,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
